In [19]:
import numpy as np
import pandas as pd
import scipy.io
import time
import timeit

In [21]:
#------------------    Read in and Process Real Data    ----------------------#
# Read 311 data
t1 = time.time()
requests = pd.read_csv('../../311__Service_Requests.csv')
t2 = time.time()
print "Read in data in %.2f seconds." % (t2 - t1)

# Filter for closed cases
t1 = time.time()
closed_requests = requests[requests['CASE_STATUS'] == 'Closed']
t2 = time.time()
print "Filtered data in %.2f seconds." % (t2 - t1)

# Convert dates to datetime type
t1 = time.time()
for col in ['OPEN_DT', 'TARGET_DT', 'CLOSED_DT']:
    closed_requests[col] = pd.to_datetime(closed_requests[col], infer_datetime_format=True)
t2= time.time()
print "Dates processed in %.2f seconds." % (t2 - t1)

Read in data in 10.75 seconds.
Filtered data in 0.30 seconds.

/Users/isadoranun/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/isadoranun/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Dates processed in 461.30 seconds.


In [31]:
#---------------- Plot Cluster Membership Histogram ----------------#

def filter_by_id(ids, source):
    begin = pd.to_datetime('January 01, 2015 12:00AM') 
    end = pd.to_datetime('January 01, 2016 12:00AM')

    # Filter for requests opened between begin and end dates
    in_range = closed_requests[closed_requests['OPEN_DT'] > begin]
    in_range = in_range[in_range['OPEN_DT'] < end]

    # Filter for requests with specific source
    in_range = in_range[in_range['Source'] == source]
    
   
    # Filter for requests for a set of case ids
    in_range = in_range[in_range['CASE_ENQUIRY_ID'].isin(ids)]
    
    return in_range

In [71]:
clusters_calls = scipy.io.loadmat("2015_data_const_MAP_ids.mat")
clusters_apps = scipy.io.loadmat("2015_data_app_MAP_ids.mat")

In [72]:
ids0_calls = clusters_calls['0'].tolist()[0]
ids1_calls = clusters_calls['1'].tolist()[0]
ids2_calls = clusters_calls['2'].tolist()[0]

ids0_apps = clusters_apps['0'].tolist()[0]
ids1_apps = clusters_apps['1'].tolist()[0]
ids2_apps = clusters_apps['2'].tolist()[0]

In [87]:
a0_calls = filter_by_id(ids0_calls,'Constituent Call')
a0_calls = a0_calls[['LONGITUDE','LATITUDE','Department','SUBJECT','REASON']]
a0_calls['Cluster'] = np.zeros(len(a0_calls))
a1_calls = filter_by_id(ids1_calls,'Constituent Call')
a1_calls = a1_calls[['LONGITUDE','LATITUDE','Department','SUBJECT','REASON']]
a1_calls['Cluster'] = np.ones(len(a1_calls))
a2_calls = filter_by_id(ids2_calls,'Constituent Call')
a2_calls = a2_calls[['LONGITUDE','LATITUDE','Department','SUBJECT','REASON']]
a2_calls['Cluster'] = 2*np.ones(len(a2_calls))

a0_apps = filter_by_id(ids0_apps,'Citizens Connect App')
a0_apps = a0_apps[['LONGITUDE','LATITUDE','Department','SUBJECT','REASON']]
a0_apps['Cluster'] = np.zeros(len(a0_apps))
a1_apps = filter_by_id(ids1_apps,'Citizens Connect App')
a1_apps = a1_apps[['LONGITUDE','LATITUDE','Department','SUBJECT','REASON']]
a1_apps['Cluster'] = np.ones(len(a1_apps))
a2_apps = filter_by_id(ids2_apps,'Citizens Connect App')
a2_apps = a2_apps[['LONGITUDE','LATITUDE','Department','SUBJECT','REASON']]
a2_apps['Cluster'] = 2*np.ones(len(a2_apps))

In [76]:
print len(a0_apps),len(a1_apps),len(a2_apps)

15771 2566 136


In [88]:
all_data_calls = pd.concat([a1_calls,a0_calls,a2_calls])
all_data_apps = pd.concat([a0_apps,a1_apps,a2_apps])

In [89]:
all_data_calls = all_data_calls.reset_index()
all_data_apps = all_data_apps.reset_index()

In [90]:
all_data_calls.to_json("clusters_calls.json", orient="records")
all_data_apps.to_json("clusters_apps.json", orient="records")

In [86]:
filter_by_id(ids0_calls,'Constituent Call').columns

Index([u'CASE_ENQUIRY_ID', u'OPEN_DT', u'TARGET_DT', u'CLOSED_DT',
       u'OnTime_Status', u'CASE_STATUS', u'CLOSURE_REASON', u'CASE_TITLE',
       u'SUBJECT', u'REASON', u'TYPE', u'QUEUE', u'Department',
       u'SubmittedPhoto', u'ClosedPhoto', u'Location', u'fire_district',
       u'pwd_district', u'city_council_district', u'police_district',
       u'neighborhood', u'neighborhood_services_district', u'ward',
       u'precinct', u'land_usage', u'LOCATION_STREET_NAME',
       u'LOCATION_ZIPCODE', u'Property_Type', u'Property_ID', u'LATITUDE',
       u'LONGITUDE', u'Source', u'Geocoded_Location'],
      dtype='object')